# 03 — Rule-Based Hand Gestures

This notebook introduces a **rule-based gesture system** on top of MediaPipe
hand landmarks.

Goals:
- Isolate **gesture detection rules** (conditions)
- Isolate **actions** (what happens when a gesture is detected)
- Make the system **easy to extend** (add rules / actions without touching the core loop)

This is the foundation for:
- Keyboard control
- Gamepad control
- Custom interactions

### Imports

Libraries required for:
- Webcam access and visualization
- MediaPipe Tasks API (Hands Landmarker)
- Landmark geometry analysis

In [ ]:
import cv2
import time
import math

import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

from pathlib import Path

### Hand Landmarker setup

Downloads the MediaPipe hand landmark model if needed  
and initializes the Hands detector (Tasks API).

In [ ]:
MODEL_PATH = Path("hand_landmarker.task")

if not MODEL_PATH.exists():
    !wget -O hand_landmarker.task https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task
else:
    print("hand_landmarker.task already exists")

base_options = python.BaseOptions(
    model_asset_path=str(MODEL_PATH)
)

options = vision.HandLandmarkerOptions(
    base_options=base_options,
    num_hands=2,
    min_hand_detection_confidence=0.7,
    min_hand_presence_confidence=0.7
)

detector = vision.HandLandmarker.create_from_options(options)

print("MediaPipe Tasks (Hands) successfully initialized!")


### Utility Functions

Helper functions used across gesture rules.
They must remain **simple and reusable**.

In [ ]:
def get_distance(p1, p2):
    """Euclidean distance between two normalized landmarks."""
    return math.sqrt((p1.x - p2.x)**2 + (p1.y - p2.y)**2)

### Gesture Rules (Conditions Only)

Each rule:
- Takes hand landmarks as input
- Returns True / False
- Contains **no side effects**

In [ ]:
def is_pinch(landmarks, threshold=0.05):
    """Thumb tip close to index tip."""
    return get_distance(landmarks[4], landmarks[8]) < threshold


def is_open_hand(landmarks, threshold=0.08):
    """All fingertips far from palm center."""
    palm = landmarks[0]
    fingertips = [4, 8, 12, 16, 20]
    return all(get_distance(landmarks[t], palm) > threshold for t in fingertips)


### Actions

Actions are triggered when a gesture rule is satisfied.

In [ ]:
def action_pinch(hand_label):
    print(f"🤏 Pinch detected on {hand_label} hand")


def action_open_hand(hand_label):
    print(f"✋ Open hand detected on {hand_label} hand")


### Gesture → Action Mapping

This table links:
- a gesture rule
- to an action

Adding a new gesture = **one line**.

In [ ]:
GESTURE_RULES = [
    ("Pinch", is_pinch, action_pinch),
    ("OpenHand", is_open_hand, action_open_hand),
]

### Real-Time Camera Loop

- Captures webcam frames
- Runs hand detection
- Applies gesture rules
- Executes matching actions

Press **q** to quit.

In [ ]:
HAND_CONNECTIONS = [
    (0, 1), (1, 2), (2, 3), (3, 4),             # Thumb
    (0, 5), (5, 6), (6, 7), (7, 8),             # Index finger
    (9, 10), (10, 11), (11, 12),                # Middle finger
    (13, 14), (14, 15), (15, 16),               # Ring finger
    (0, 17), (17, 18), (18, 19), (19, 20),      # Pinky
    (5, 9), (9, 13), (13, 17)                   # Palm
]

# Camera initialization
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Cannot open camera")

try:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to read frame")
            break

        # Mirror view for a more natural interaction
        frame = cv2.flip(frame, 1)
        h, w, _ = frame.shape

        # Convert frame to RGB (required by MediaPipe)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        mp_image = mp.Image(
            image_format=mp.ImageFormat.SRGB,
            data=rgb_frame
        )

        # Hand landmark detection
        result = detector.detect(mp_image)

        if result.hand_landmarks:
            for hand_idx, landmarks in enumerate(result.hand_landmarks):
                
                # 1. Draw hand connections (skeleton)
                for start_idx, end_idx in HAND_CONNECTIONS:
                    start_pt = (
                        int(landmarks[start_idx].x * w),
                        int(landmarks[start_idx].y * h)
                    )
                    end_pt = (
                        int(landmarks[end_idx].x * w),
                        int(landmarks[end_idx].y * h)
                    )
                    cv2.line(frame, start_pt, end_pt, (0, 255, 0), 2)
                
                # 2. Draw landmark points
                for lm in landmarks:
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)

                hand_label = result.handedness[hand_idx][0].category_name

                # # 3 Apply gesture rules
                for name, rule_fn, action_fn in GESTURE_RULES:
                    if rule_fn(landmarks):
                        action_fn(hand_label)

        # Display the camera feed
        cv2.imshow("MediaPipe Hands", frame)

        # Exit on 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    cap.release()
    cv2.destroyAllWindows()
    print("Camera closed cleanly")
